In [3]:
import re
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



In [30]:

#
# Parse StarOS Show command output from multiple VNFs
#
# File should contain: name of VNF, show system uptime, show crt stat, show version, show subscribers summary
#  and separated by 69 char repeating =
# Ex:
#    =====================================================================
#    ATPGW100
#    [u'Wednesday September 05 16:31:14 UTC 2018
#    Active Software:
#      Image Version:                  21.6.11
#      Image Build Number:             69836
#    ...
############
with open('all_090518_staros_show_vers_uptime_rct_crash.txt', 'r') as f:
    rawc = f.read()

# Contains each unique VNF
unique_vnf_data = []

# Contains each VNF crash
crash_list_data = []

for chk in re.split('\={69}', rawc):
    r = {}
    m = re.search('([A-Z]{5}1[0-9]{2})', chk)
    if m:
        r['vnf'] = m.group(1)
    m = re.search('Image Version: +(.*)', chk)
    if m:
        r['version'] = m.group(1)
    m = re.search('Total Subscribers: +([0-9]+)', chk)
    if m:
        r['total_subs'] = m.group(1)
    else:
        r['total_subs'] = '0'
    m = re.search('System uptime:? ([0-9]+)D? ([0-9]+)H? ([0-9]+)M', chk)
    if m:
        r['uptime_days'] = int(m.group(1)) + 1
    m = re.search('[A-Z][a-z]+ ([A-Z][a-z]+ [0-9]{2} [0-9]{2}:[0-9]{2}:[0-9]{2} )[A-Z]{3} ([0-9]{4})', chk)
    if m:
        r['system_time'] = datetime.strptime(m.group(1) + m.group(2), '%B %d %H:%M:%S %Y')
    r['rct_stat_count'] = 0
    # Build rct list
    if re.search('RCT stats details', chk):
        sp_crs = re.split('RCT stats details|^$', chk)
        for sp_crr in sp_crs[1].split('\n'):
            #if re.search('Success', sp_crr) and re.search('(Aug|Sep)', sp_crr) and not re.search('21.1.v7', cr['version']):
            if re.search('Success', sp_crr) and not re.search('21.1.v7', r['version']):
                sp_sp_crr = re.split(' +', sp_crr[2:])
                cr = {}
                cr = {
                    'type': sp_sp_crr[1], #[4] because blank space if <10 count in list
                    'reason': sp_sp_crr[2],
                    'from_card': sp_sp_crr[3],
                    'to_card': sp_sp_crr[4],
                    'crashtime': datetime.strptime(sp_sp_crr[5][:-4], '%Y-%b-%d+%H:%M:%S'),
                    'vnf': r['vnf'],
                    'version': r['version'],
                    'total_subs': r['total_subs'],
                    'uptime_days': r['uptime_days'],
                    'system_time': r['system_time'],
                }
                cr['last_reboot'] = cr['system_time'] - timedelta(days=cr['uptime_days'])
                crash_boot_diff = cr['crashtime'] - cr['last_reboot']
                cr['crash_lastboot_time_diff'] = crash_boot_diff.days
                aug = datetime.now() - timedelta(days=40)
                if re.search('[1-2]', cr['from_card']) and cr['reason'] != 'Planned' and cr['crashtime'] > aug:
                    crash_list_data.append(cr)
                    r['rct_stat_count'] += 1
                    #print(cr)
    if r.get('vnf', None):
        unique_vnf_data.append(r)

for i in unique_vnf_data:
    print(i['vnf'] + ',' + str(i['uptime_days']))


ATPCF100,14
ATPCF101,163
ATPGW100,36
ATPGW101,163
ATPGW102,41
ATPGW103,37
ATPGW104,10
ATPGW105,7
ATPGW106,36
AUPCF100,15
AUPGW100,15
AUPGW101,182
AUPGW102,182
BHPCF100,11
BHPGW100,28
CBPCF100,14
CBPGW100,14
CHPCF100,42
CHPCF101,38
CHPGW100,42
CHPGW101,38
CHPGW102,45
CHPGW103,45
CHPGW104,45
CHPGW105,44
CHPGW106,156
CHPGW107,156
CHPGW108,44
CHPGW109,44
CHPGW110,150
CLPCF100,43
CLPGW100,43
CRPGW100,37
CRPGW101,45
CRPGW102,45
CRPGW103,45
CRPGW104,142
CRPGW105,45
CRPGW106,45
CRPGW107,142
CRPGW108,142
CRPGW109,41
CTPCF100,14
CTPGW100,14
DAPCF100,24
DAPCF101,22
DAPGW100,24
DAPGW101,22
DAPGW102,41
DAPGW103,147
DAPGW104,35
DAPGW105,14
DAPGW106,10
DAPGW107,8
DAPGW108,7
DAPGW109,8
DAPGW110,7
DAPGW111,42
DCPCF100,15
DCPGW100,15
DCPGW101,44
DCPGW102,122
DCPGW103,122
DCPGW104,35
DCPGW105,41
DCPGW106,146
DEPCF100,1
DEPCF101,8
DEPGW100,10
DEPGW101,8
DEPGW102,141
DEPGW103,78
DEPGW104,146
DEPGW105,41
DNPCF100,68
DNPGW100,68
DNPGW101,121
DNPGW102,1
DNPGW103,1
DNPGW104,122
DNPGW105,122
FLPCF100,21
FLPCF10

/opt/conda/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [26]:
#
# Put into pandas data
######################

df = pd.DataFrame(crash_list_data)
#df['crashtime'] = pd.to_datetime(df['crashtime'])
#df = df.set_index('crashtime')
df

#df.drop(['crashtime'], axis=1, inplace=True)
#df.groupby(['crash_lastboot_time_diff']).mean()
#df.plot.bar(figsize=(25,25), fontsize=25)
#df.plot.bar(fontsize=25)
#plt.scatter(df['uptime_days'], df['crash_lastboot_time_diff'])
#plt.show()


,crash_lastboot_time_diff,crashtime,from_card,last_reboot,reason,system_time,to_card,total_subs,type,uptime_days,version,vnf
0,30,2018-08-31 03:10:26,2,2018-07-31 16:31:17,N/A,2018-09-05 16:31:17,0,266110,Shutdown,36,21.6.11,ATPGW106
1,32,2018-09-01 17:01:19,2,2018-07-31 16:31:17,N/A,2018-09-05 16:31:17,0,266110,Shutdown,36,21.6.11,ATPGW106
2,32,2018-09-02 00:49:16,2,2018-07-31 16:31:17,N/A,2018-09-05 16:31:17,0,266110,Shutdown,36,21.6.11,ATPGW106
3,35,2018-09-05 09:48:05,2,2018-07-31 16:31:17,N/A,2018-09-05 16:31:17,0,266110,Shutdown,36,21.6.11,ATPGW106
4,0,2018-08-22 09:21:19,1,2018-08-21 16:31:17,Unplanned,2018-09-05 16:31:17,2,855644,Switchover,15,21.6.11,AUPCF100
5,2,2018-08-25 09:50:30,2,2018-08-22 16:31:23,N/A,2018-09-05 16:31:23,0,348674,Shutdown,14,21.6.11,CBPCF100
6,3,2018-08-25 22:03:51,2,2018-08-22 16:31:23,N/A,2018-09-05 16:31:23,0,348674,Shutdown,14,21.6.11,CBPCF100
7,4,2018-08-26 21:11:34,2,2018-08-22 16:31:23,N/A,2018-09-05 16:31:23,0,348674,Shutdown,14,21.6.11,CBPCF100
8,40,2018-09-04 08:20:08,1,2018-07-25 16:31:23,Unplanned,2018-09-05 16:31:23,2,714285,Switchover,42,21.6.11,CHPCF100
9,12,2018-08-03 19:23:49,2,2018-07-22 16:31:26,N/A,2018-09-05 16:31:26,0,0,Shutdown,45,21.6.11,CHPGW102
